# Crawler

This notebook contains started code structure for creating a crawler on single machine

**Author:** Noshaba Nasir

**Date:** 26/3/2021

**Updated by:** 17L4545
    

In [3]:
import os 
import random
import urllib.parse
import datetime
import time
import requests
import bs4
import _thread
import urllib.robotparser as urobot
import urllib.request
from bs4 import BeautifulSoup
from urllib.request import urljoin





In [4]:
# Crawler Parameters
BACKQUEUES= 3
THREADS= BACKQUEUES*3
FRONTQUEUES= 5
WAITTIME= 15 ; # wait 15 seconds before fetching URLS from 

# Add any other global parameters here

# FRONTIER

Frontier should use the Mercator frontier design as discussed in lecture.

Preferably it should be a class and should have the given functions.

*prioritizer* function is a stub right now, it will return a random number  between 1 to f for given URL 

In [5]:
class frontier:
# add the code for frontier here
    def __init__(self,urls):
        self.frontqueues = [[] for _ in range(FRONTQUEUES)]
        self.backqueues = [[] for _ in range(BACKQUEUES)]
        self.add_URLs(urls)
        i = 0
        while i <=8:
            self.add_to_backqueue()
            i=i+1
        self.timestamps=[]
        None # replace this with your code
        
    # add more functions here
    
    def prioritizer(self,URL,f):
        return random.randint(0,f-1)
    
    def add_URLs(self,urls):
        for url in urls:
            priority=self.prioritizer(urls,FRONTQUEUES)
            #print(priority)
            self.frontqueues[priority].append(url)
            
            
    def add_to_backqueue(self):
        
        #extracts a url from front queues on base of priority probability
        url=''
        qno=0    #to store no of front q from which url is taken
        prob=random.randint(1,14)
        if(prob<5 and len(self.frontqueues[0])!=0):
            url=self.frontqueues[0].pop(0)  #pop from frontq 0
            qno=0
        elif(prob<9 and len(self.frontqueues[1])!=0):
            url=self.frontqueues[1].pop(0)  #pop from frontq 0
            qno=1
        elif(prob<12 and len(self.frontqueues[2])!=0):
            url=self.frontqueues[2].pop(0)  #pop from frontq 0
            qno=2
        elif(prob<14 and len(self.frontqueues[3])!=0):
            url=self.frontqueues[3].pop(0)  #pop from frontq 0
            qno=3
        else:
            if(len(self.frontqueues[4])!=0):
                url=self.frontqueues[4].pop(0)  #pop from frontq 0
                qno=4
       
        #parsing the hostname of url extratcted from frontqueues
        parsed_url = urllib.parse.urlparse(url)
        host=parsed_url.netloc
        
        #checks if a backqueue exists with same URL host and adds provided url to that bqueue
        for bq in self.backqueues:
            if(len(bq)!=0):
                parsedurl = urllib.parse.urlparse(bq[0])
                domain=parsedurl.netloc
                if(domain==host):
                    bq.append(url)
                    return
        #checks if an empty bq exists after iterating through all bqs and checking their hosts
        for bq in self.backqueues:
            if len(bq) == 0:
                bq.append(url)
                return
        self.frontqueues[qno].append(url)  #add to frontqueue back if not added to any backqueue
        

     
    def get_URL(self):
        #init for timestamps list equal to bqs in size
        url=''
        if(len(self.timestamps)==0):
            i=0
            while i <BACKQUEUES:
                self.timestamps.append(datetime.datetime.utcnow())    
                time.sleep(1)
                i=i+1
        
        min_index = self.timestamps.index(min(self.timestamps))
        if(self.timestamps[min_index]< datetime.datetime.utcnow()):
            if(len(self.backqueues[min_index])!=0):
                url=self.backqueues[min_index].pop(0)
                return url
                added_seconds = datetime.timedelta(0, 15)
                self.timestamps[min_index]=datetime.datetime.utcnow()+added_seconds
            return url
    
   


        
#Helper functions

#function to return all urls embedded on a html page
def fetch(url):
    resp = requests.get(url)
    with open('file.xml', 'wb') as foutput:
        foutput.write(resp.content)
    foutput.close()

    links_arr=[]
    parser = 'html.parser'  
    resp = urllib.request.urlopen(url)
    soup = BeautifulSoup(resp, parser, from_encoding=resp.info().get_param('charset'))
    for link in soup.find_all('a', href=True):
        lnk  = link.get('href')
        if not lnk.startswith('http'):
            lnk = urljoin(url, lnk)
        #if '#' in lnk or doctype(lnk)==False :
            #continue
        if '#' in lnk:
            continue
        else:    
            links_arr.append(lnk)

    return(links_arr)

#returns true if its link of a doctype=html 
def doctype(url):
    try:
        parser = 'html.parser'  
        resp = urllib.request.urlopen(url)
        soup = BeautifulSoup(resp, parser, from_encoding=resp.info().get_param('charset'))
        items = [item for item in soup.contents if isinstance(item, bs4.Doctype)]
        if len(items)!=0:
            if items[0]=='html':
                return True
        return false
    except:
        return False
  


    



# FILTER URLS

Filter the URLS that are in robots.txt files of server and the have been already processed. 

In [6]:
#Filtering function returns true if link is allowed by robots.txt of respective site
def check_if_allowed(url):
    parsedurl = urllib.parse.urlparse(url)
    domain=parsedurl.netloc
  
    rp = urobot.RobotFileParser()
    rp.set_url("https://" +domain + "/robots.txt")
    rp.read()
    if rp.can_fetch("*", url):
        return True
    else:
        return False


## ---------------------Write any other codes/data require to run the crawler above this cell-----------

# Run Crawler

In [7]:
# Theard task
def crawler_thread_task(threadname):
    lnk=frontier_obj.get_URL()
    print(lnk)
    try:
        arr=fetch(lnk)
        for lnks in arr:
            if check_if_allowed(lnks)==False:
                links=[]
                links.append(lnks)
                frontier_obj.add_URLs(links)
                print(lnks)
            
            frontier_obj.add_to_backqueue()
        print(frontier_obj.frontqueues)
        print(threadname)
       
        
    except:
        pass
        print(threadname+"pass")
        
    

In [9]:
# intialize every thing 
listurls=['https://docs.oracle.com/en/','https://www.oracle.com/corporate/','https://en.wikipedia.org/wiki/Machine_learning','https://www.csie.ntu.edu.tw/~cjlin/libsvm/index.html','https://docs.oracle.com/middleware/jet210/jet/index.html','https://en.wikipedia.org/w/api.php','https://en.wikipedia.org/api/','https://en.wikipedia.org/wiki/Weka_(machine_learning)']
frontier_obj=frontier(listurls)
print("wait for me first before starting the thread")

wait for me first before starting the thread


In [10]:
# start the threads

try:
   _thread.start_new_thread(crawler_thread_task,("thread1",))
   _thread.start_new_thread(crawler_thread_task,("thread2",))
   _thread.start_new_thread(crawler_thread_task,("thread3",))
    
    
except:
   print("Error: unable to start thread")

https://www.oracle.com/corporate/

thread2pass

thread1pass
https://www.oracle.com/corporate/accessibility/
https://www.oracle.com/
https://www.oracle.com/universal-menu/
https://www.oracle.com/corporate/
https://www.oracle.com/cloud/sign-in.html
https://www.oracle.com/cloud/free/?source=:ow:o:h:nav:OHP0625ViewAccountsButton&intcmp=:ow:o:h:nav:OHP0625ViewAccountsButton
https://www.oracle.com/webapps/redirect/signon?nexturl=
https://profile.oracle.com/myprofile/account/create-account.jspx
https://www.oracle.com/corporate/contact/help.html
javascript:sso_sign_out();
https://www.oracle.com/corporate/contact/
https://www.oracle.com/startup/
https://www.oracle.com/corporate/careers/benefits/training.html
https://www.oracle.com/corporate/careers/culture/who-we-are.html
https://www.oracle.com/corporate/executives/
https://www.oracle.com/corporate/careers/
https://www.oracle.com/corporate/citizenship/
https://www.oracle.com/corporate/careers/culture/diversity.html
https://developer.oracle.com/

In [9]:
# start the threads in loop not working

try:
    name="thread"
    while i<THREADS:
        _thread.start_new_thread(crawler_thread_task,(name+i,))
        i=i+1
     
except:
   print("Error: unable to start thread")

Error: unable to start thread


## ------------------------------------------------------End of Notebook---------------------------------------------------